In [1]:
from tools.file_read_tool import FileReaderTool
from crew.orchestrator import FlutterTaskOrchestrator
import ipywidgets as widgets
from IPython.display import display
import nest_asyncio
from pathlib import Path
nest_asyncio.apply()

In [2]:
ROOT_FOLDER = '/home/i-b8o/src/github.com/marketconnect/mc_dashboard/lib'
file_reader = FileReaderTool(ROOT_FOLDER)


In [3]:
def list_files(directory: str, extension: str = None):
    """
    Recursively lists all files in a directory and its subdirectories.
    directory: The directory to search.
    """
    p = Path(directory)
    if extension:
        return sorted([(str(f.relative_to(directory)), str(f)) 
                       for f in p.rglob(f'*{extension}') if f.is_file()])
    else:
        return sorted([(str(f.relative_to(directory)), str(f)) 
                       for f in p.rglob('*') if f.is_file()])

In [ ]:
task_type_widget = widgets.Dropdown(
    options=['apiclient', 'service', 'viewmodel', 'screen', 'integration', 'entity', 'entity from JSON', 'question'],
    value='apiclient',
    description='Task type:',
)

task_description_widget = widgets.Textarea(
    value='',
    description='Description:',
    placeholder='Введите описание задачи...',
    layout=widgets.Layout(width='1000px', height='500px')
)

files = list_files(ROOT_FOLDER, extension=".dart")
file_dropdown = widgets.Dropdown(
    options=files,
    description='Sample File:',
    layout=widgets.Layout(width='800px')
)

new_file_name_widget = widgets.Text(
    value='',
    description='New File Name:',
    placeholder='Введите имя нового файла (например, new_entity.dart)',
    layout=widgets.Layout(width='600px')
)

entity_json_widget = widgets.Textarea(
    value='',
    description='Entity JSON:',
    placeholder='Введите JSON представление класса...',
    layout=widgets.Layout(width='800px', height='300px')
)

entity_json_widget.layout.display = 'none'

def on_task_type_change(change):
    if change.new == 'entity from JSON':
        entity_json_widget.layout.display = 'block'
    else:
        entity_json_widget.layout.display = 'none'

task_type_widget.observe(on_task_type_change, names='value')

run_button = widgets.Button(
    description='Run',
    button_style='success'
)

output_area = widgets.Output()

In [5]:
orchestrator = FlutterTaskOrchestrator(file_reader.read_file_content)

In [6]:
import asyncio


def on_run_button_clicked(b):
    with output_area:
        output_area.clear_output()
        task_type = task_type_widget.value
        task_description = task_description_widget.value
        sample_file_path = file_dropdown.value
        new_file_name = new_file_name_widget.value if task_type in ["apiclient", "service", "viewmodel", "screen", "entity", "entity from JSON"] else None
        entity_json = entity_json_widget.value if task_type == 'entity from JSON' else None

        print("Task type:", task_type)
        print("Task description:", task_description)
        print("Sample file path:", sample_file_path)
        if new_file_name:
            print("New file name:", new_file_name)
        if entity_json:
            print("Entity JSON:", entity_json)
        try:
            result = asyncio.run(orchestrator.handle_task(task_type, task_description, sample_file_path, new_file_name, entity_json))
            print("\n=== Generated Code ===")
            print(result)
        except Exception as e:
            print(f"Ошибка: {e}")

run_button.on_click(on_run_button_clicked)

In [7]:
display(task_type_widget, task_description_widget, file_dropdown, new_file_name_widget, entity_json_widget, run_button, output_area)

Dropdown(description='Task type:', options=('apiclient', 'service', 'viewmodel', 'screen', 'integration', 'ent…

Textarea(value='', description='Description:', layout=Layout(height='500px', width='1000px'), placeholder='Вве…

Dropdown(description='Sample File:', layout=Layout(width='800px'), options=(('.env.dart', '/home/i-b8o/src/git…

Text(value='', description='New File Name:', layout=Layout(width='600px'), placeholder='Введите имя нового фай…

Textarea(value='', description='Entity JSON:', layout=Layout(display='none', height='300px', width='800px'), p…

Button(button_style='success', description='Run', style=ButtonStyle())

Output()